## Same as sample dataset,

but this return not truncated data.

just for fast validation.

inference need to do all of these in that notebook.

In [1]:
N_DAYS_PER_SAMPLE = 1
FIT_INTO_1DAY = True
SEPERATE_FEATURES = True  # Not Used
# GENERATE_FEATURES = True # generate in Train torch.data.DataSet for efficiency

SIGMA = 720 # target gaussian parameter, 12 * 60

In [2]:
import pandas as pd
import numpy as np
import gc
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os
from os.path import join, exists
import joblib
import random
import math
from tqdm.auto import tqdm 
import shutil

from scipy.interpolate import interp1d

from math import pi, sqrt, exp
import sklearn,sklearn.model_selection
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa 
import ctypes

In [3]:
out_dir = "train_data"
os.makedirs(out_dir, exist_ok = True)

In [4]:
class PATHS:
    MAIN_DIR = "/kaggle/input/child-mind-institute-detect-sleep-states/"
    # CSV FILES : 
    TRAIN_EVENTS = MAIN_DIR + "train_events.csv"
    # PARQUET FILES:
    TRAIN_SERIES = MAIN_DIR + "train_series.parquet"
    SAVE_DIR = out_dir

In [5]:
def preprocess_events(data_path, save_dir):
    """ events.csv """
    events = pd.read_csv(data_path)
    if not exists(join(save_dir, 'id_map.parquet')):
        id_map = pd.DataFrame({"series_id": events.series_id.unique(),
                                "id_index": [i for i in range(len(events.series_id.unique()))]})
        id_map.id_index = id_map.id_index.astype(np.uint16)
        id_map.to_parquet(os.path.join(save_dir,"id_map.parquet"), index=False)
    id_map = pd.read_parquet(os.path.join(save_dir,"id_map.parquet"))
    events.dropna(subset=['step'], inplace=True)
    # reduce memory size + event re-label
    events = events.merge(right=id_map, on="series_id").drop(columns="series_id")
    events.step = events.step.astype(np.uint32)
    events.night = events.night.astype(np.uint16)
#     events.to_csv(os.path.join(save_dir,'preprocessed_events.csv'), index=False)
    
    return events

In [6]:
def preprocess_series(data_path, save_dir):
    """ train_series.parquet """
    # reduce memory
    series = pd.read_parquet(data_path)
    if not exists(join(save_dir, 'id_map.parquet')):
        id_map = pd.DataFrame({"series_id": series.series_id.unique(),
                                "id_index": [i for i in range(len(series.series_id.unique()))]})
        id_map.id_index = id_map.id_index.astype(np.uint16)
        id_map.to_parquet(os.path.join(save_dir,"id_map.parquet"), index=False)
    
    id_map = pd.read_parquet(os.path.join(save_dir,"id_map.parquet"))
    series = series.merge(right=id_map, on='series_id').drop(columns='series_id').reset_index()
    series.step = series.step.astype(np.uint32)

    # normalize enmo and anglez
    mean_enmo = series['enmo'].mean()
    std_enmo = series['enmo'].std()
    series.enmo = (series['enmo'] - mean_enmo)/std_enmo
    mean_anglez = series['anglez'].mean()
    std_anglez = series['anglez'].std()
    series.anglez = (series['anglez'] - mean_anglez)/std_anglez
    
    series = series.drop(columns='index')
#     series.to_parquet(os.path.join(save_dir,'preprocessed_series.parquet'))
    
    return series

In [7]:
events = preprocess_events(PATHS.TRAIN_EVENTS, PATHS.SAVE_DIR)
series = preprocess_series(PATHS.TRAIN_SERIES, PATHS.SAVE_DIR).drop(columns=['timestamp','step'])
ids = pd.read_parquet(os.path.join(PATHS.SAVE_DIR,"id_map.parquet"))

# store series with target gaussian

- raw form with preprocessed, target gaussian

In [8]:
def gauss(n=SIGMA,sigma=SIGMA*0.15):
    # guassian distribution function
    r = range(-int(n/2),int(n/2)+1)  # int는 나머지 버림계산
    return [1 / (sigma * sqrt(2*pi)) * exp(-float(x)**2/(2*sigma**2)) for x in r]


for cur_id in tqdm(ids.id_index, total=len(ids)):
    cur_events = events[events.id_index == cur_id].copy()
    cur_series = cur_series=series.loc[series.id_index==cur_id].copy().reset_index(drop=True).drop(columns='id_index')
    
    # make gaussian
    target_gaussian = np.zeros((len(cur_series),2))
    for i in range(len(cur_events)):
        s = cur_events.iloc[i].step
        tar_st=s-SIGMA//2
        tar_ed=s+SIGMA//2
        if tar_st<0:
            gau_st = -(SIGMA//2-s)
            tar_st = 0
        else:
            gau_st=0
        if tar_ed>=len(cur_series):
            residual = tar_ed-(len(cur_series)-1)
            gau_ed=SIGMA-residual
            tar_ed=len(cur_series)-1
        else:
            gau_ed=SIGMA
        
#         if len(target_gaussian[tar_st:tar_ed,0])!=len(gauss()[gau_st:gau_ed]):
#             print(tar_st, tar_ed)
#             print(gau_st, gau_ed)
#             print(len(target_gaussian[tar_st:tar_ed,0]))
#             print(len(gauss()[gau_st:gau_ed]))
        if cur_events.iloc[i].event=='onset':
            target_gaussian[tar_st:tar_ed,0] = gauss()[gau_st:gau_ed]
        if cur_events.iloc[i].event=='wakeup':
            target_gaussian[tar_st:tar_ed,1] = gauss()[gau_st:gau_ed]
    # normalize target into max value.
    target_gaussian /= np.max(target_gaussian + 1e-12)
    # append target onto series,
    cur_series['onset'] = target_gaussian[:,0]
    cur_series['wakeup'] = target_gaussian[:,1]
    # save it.
    cur_series.to_csv(f'{PATHS.SAVE_DIR}/id{cur_id}.csv', index=False)

  0%|          | 0/277 [00:00<?, ?it/s]

# save

In [9]:
os.makedirs('./archive', exist_ok=True)
shutil.make_archive(f'./archive/data', 'zip', f'{PATHS.SAVE_DIR}')
shutil.move(f'./{PATHS.SAVE_DIR}/id_map.parquet', './archive')
shutil.rmtree(PATHS.SAVE_DIR)


# Creating a dictionary with dummy data for testing
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emma'],
    'Age': [25, 30, 35, 40, 45],
    'City': ['New York', 'San Francisco', 'Los Angeles', 'Chicago', 'Seattle']
}

a = pd.DataFrame(data)
a.to_csv('./archive/dummy.csv')